# 最大似然估计学习

onlytiancai 2025-08

1. 生成随机数据并可视化
2. 最大似然估计的数学推导及直接求解析解
3. 理解概率密度和联合概率密度
4. 梯度下降法求最大似然
5. 什么情况下不能用最大似然估计

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']  # Windows 黑体，如果在 Mac 可换成 ['PingFang SC']
plt.rcParams['axes.unicode_minus'] = False   # 解决负号显示问题

## 一、生成随机数据并可视化

In [ ]:
mu_true = 5.0
sigma_true = 2.0
n_samples = 1000

np.random.seed(42)
data = np.random.normal(loc=mu_true, scale=sigma_true, size=n_samples)

plt.figure(figsize=(18, 5))

# 直方图
plt.subplot(1, 3, 1)
plt.hist(data, bins=30, density=True, alpha=0.6, color='b')
plt.title('Histogram')
plt.xlabel('Value')
plt.ylabel('Density')
plt.grid(True)

# KDE 图
plt.subplot(1, 3, 2)
sns.kdeplot(data, fill=True, color='purple')
plt.title('Kernel Density Estimation (KDE)')
plt.xlabel('Value')
plt.ylabel('Density')

# Boxplot 图
plt.subplot(1, 3, 3)
sns.boxplot(x=data, color='orange')
plt.title('Boxplot')
plt.xlabel('Value')

plt.tight_layout()
plt.show()



### 1. **直方图（Histogram）**

* **原理**：
  把数据的取值范围分成很多小区间（称为“桶”或“bin”），统计每个区间内数据点的数量。然后用矩形条形表示每个区间的频数（或频率、密度）。
* **作用**：
  展示数据的分布形状（比如是否偏态、是否有多峰），直观体现数据在哪些值附近密集。
* **缺点**：
  结果对区间宽度和起点比较敏感，不同的分箱方式可能看起来差别很大。

---

### 2. **核密度估计图（KDE, Kernel Density Estimation）**

* **原理**：
  KDE 是一种平滑的概率密度估计方法。它在每个数据点上放一个小“核函数”（通常是高斯核），然后把这些核函数叠加起来，形成一个连续的平滑曲线，估计数据的概率密度函数。
* **作用**：
  更平滑地展示数据分布，避免了直方图分箱带来的不连续感，可以更清楚看到数据的整体趋势和潜在的多峰结构。
* **缺点**：
  KDE 的平滑程度取决于带宽（kernel bandwidth）参数，选择不当会导致过于平滑或过于嘈杂。

---

### 3. **箱型图（Boxplot）**

* **原理**：
  箱型图通过五数概括（最小值、第一四分位数Q1、中位数Q2、第三四分位数Q3、最大值）来总结数据的分布。盒子显示中间50%的数据范围（Q1到Q3），中间的线表示中位数，盒子外的“须”通常表示数据的变动范围，超出“须”的点被认为是异常值。
* **作用**：
  快速显示数据的集中趋势、离散程度、偏态及异常值，非常适合做不同组数据间的比较。
* **缺点**：
  只显示统计摘要，不能像直方图或KDE那样详细展示数据的整体形态。



# 二、最大似然估计的数学推导及直接求解析解

**正态分布 $X_i\overset{\text{iid}}{\sim}N(\mu,\sigma^2)$** 的最大似然估计（MLE）详细推导，逐步求导并检验极值性质。

## 1. 写出似然函数与对数似然

样本为 $x_1,\dots,x_n$。联合密度（似然）为

$$
L(\mu,\sigma^2)=\prod_{i=1}^n \frac{1}{\sqrt{2\pi\sigma^2}}\exp\Big(-\frac{(x_i-\mu)^2}{2\sigma^2}\Big).
$$

取对数得对数似然

$$
\ell(\mu,\sigma^2)=\log L(\mu,\sigma^2)
= -\frac{n}{2}\log(2\pi\sigma^2)-\frac{1}{2\sigma^2}\sum_{i=1}^n (x_i-\mu)^2.
$$

## 2. 对 $\mu$ 求偏导并令其为 0

对 $\mu$ 求偏导：

$$
\frac{\partial \ell}{\partial \mu}
= -\frac{1}{2\sigma^2}\cdot 2\sum_{i=1}^n (x_i-\mu)(-1)
= \frac{1}{\sigma^2}\sum_{i=1}^n (x_i-\mu).
$$

令其为 0：

$$
\sum_{i=1}^n (x_i-\mu)=0 \quad\Rightarrow\quad n\mu=\sum_{i=1}^n x_i.
$$

于是得到

$$
\hat\mu_{\text{MLE}}=\bar x=\frac{1}{n}\sum_{i=1}^n x_i.
$$

（可检验二阶导数：$\frac{\partial^2\ell}{\partial\mu^2}=-\frac{n}{\sigma^2}<0$，为极大值。）

## 3. 对 $\sigma^2$ 求偏导并令其为 0

先对 $\sigma^2$ 求偏导（把 $\mu$ 用上一行的 $\hat\mu$ 替代或先一般求导再代入）：

$$
\frac{\partial \ell}{\partial \sigma^2}
= -\frac{n}{2}\cdot\frac{1}{\sigma^2} + \frac{1}{2(\sigma^2)^2}\sum_{i=1}^n (x_i-\mu)^2.
$$

令其为 0 得

$$
-\frac{n}{2\sigma^2} + \frac{1}{2(\sigma^2)^2}\sum_{i=1}^n (x_i-\mu)^2 = 0.
$$

两边乘以 $2(\sigma^2)^2$：

$$
-n\sigma^2 + \sum_{i=1}^n (x_i-\mu)^2 = 0
$$

解得

$$
\hat\sigma^2_{\text{MLE}}=\frac{1}{n}\sum_{i=1}^n (x_i-\mu)^2.
$$

通常把 $\mu$ 用其 MLE $\bar x$ 代入，所以

$$
\boxed{\;\hat\mu_{\text{MLE}}=\bar x,\qquad \hat\sigma^2_{\text{MLE}}=\frac{1}{n}\sum_{i=1}^n (x_i-\bar x)^2\;}
$$

## 4. 关于有无偏性的补充

* $\hat\mu_{\text{MLE}}=\bar x$ 是无偏且是样本均值的常见估计。
* $\hat\sigma^2_{\text{MLE}}$（除以 $n$）**是有偏的**：其期望为 $\mathbb{E}[\hat\sigma^2_{\text{MLE}}]=\frac{n-1}{n}\sigma^2$。若要无偏估计方差，应使用

  $$
  s^2=\frac{1}{n-1}\sum_{i=1}^n (x_i-\bar x)^2,
  $$

  这是通常在样本方差中采用的 $n-1$ 分母（贝塞尔校正）。


## 5、打印似然估计值

In [ ]:
mu_mle = np.mean(data)
sigma2_mle = np.mean((data - mu_mle)**2)
sigma_mle = np.sqrt(sigma2_mle)

print(mu_mle, sigma2_mle, sigma_mle)

## 6、可视化估计值，真实值

In [ ]:
# 4. 绘制直方图 + PDF 曲线
plt.figure(figsize=(8,5))
# 数据直方图
count, bins, _ = plt.hist(data, bins=30, density=True, alpha=0.6, color='skyblue', label='随机数据直方图')

# x 轴范围
x = np.linspace(min(data), max(data), 200)

# 真实分布 PDF
plt.plot(x, norm.pdf(x, mu_true, sigma_true), 'r-', lw=2, label='真实分布 PDF')

# MLE 估计的 PDF
plt.plot(x, norm.pdf(x, mu_mle, sigma_mle), 'g--', lw=2, label='MLE 估计 PDF')

plt.title('正态分布随机点及其 MLE 拟合', fontsize=14)
plt.xlabel('值')
plt.ylabel('概率密度')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

# 三、理解概率密度和联合概率密度

## 1、概率密度

In [ ]:
from scipy.stats import norm, multivariate_normal

# ======================
# 1. 一维概率密度
# ======================
x = np.linspace(-4, 4, 200)
mu = 0
sigma = 1
pdf_1d = norm.pdf(x, mu, sigma)

计算标准正态分布（均值0，标准差1）在一组点上的概率密度函数（PDF）值。

```python
x = np.linspace(-4, 4, 200)
```

* 这行代码生成了一个包含200个点的一维数组 `x`，这些点均匀地分布在 -4 到 4 的区间内。
* 这些点可以看作我们要计算概率密度的自变量（横轴坐标）。

```python
mu = 0
sigma = 1
```

* 定义了正态分布的参数：均值（`mu`）为0，标准差（`sigma`）为1。
* 这代表我们用的是标准正态分布。

```python
pdf_1d = norm.pdf(x, mu, sigma)
```

* 使用 `scipy.stats.norm.pdf` 函数计算正态分布在 `x` 中每个点对应的概率密度函数值（PDF）。
* 结果是一个数组 `pdf_1d`，包含了每个 `x` 点对应的概率密度。
* 这些值可以用来画出正态分布的曲线。

这三行代码生成了一组点 `x`，然后计算了标准正态分布在这些点上的概率密度。


In [ ]:
plt.plot(x, pdf_1d, 'b-', lw=2)
plt.title("一维正态分布概率密度", fontsize=14)
plt.xlabel("x")
plt.ylabel("概率密度 f(x)")
plt.grid(alpha=0.3)
plt.show()

## 2、联合概率密度

In [ ]:
mu_vec = np.array([0, 0])
cov_mat = np.array([[1, 0.5],
                    [0.5, 1]])

X, Y = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))
pos = np.dstack((X, Y))
rv = multivariate_normal(mean=mu_vec, cov=cov_mat)
pdf_2d = rv.pdf(pos)

```python
mu_vec = np.array([0, 0])
cov_mat = np.array([[1, 0.5],
                    [0.5, 1]])
```

* 定义二维正态分布的参数：

  * 均值向量 `mu_vec` 是 `[0, 0]`，表示两个维度的均值都是0。
  * 协方差矩阵 `cov_mat` 是一个2x2矩阵，表示两个维度的方差和协方差。对角线元素1表示两个维度的方差都是1，非对角线的0.5表示两个维度之间的正相关。

---

```python
X, Y = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))
```

* 生成一个二维网格坐标矩阵：

  * `np.linspace(-3, 3, 100)` 在区间 -3 到 3 内生成100个点。
  * `np.meshgrid` 根据这两个一维数组生成二维网格的X和Y坐标矩阵，分别存储每个网格点的横纵坐标。

---

```python
pos = np.dstack((X, Y))
```

* 把两个二维坐标矩阵 `X` 和 `Y` 按最后一个维度堆叠起来，形成一个形状为 `(100, 100, 2)` 的数组 `pos`。
* 每个元素 `pos[i,j]` 是一个二维点的坐标 `[x, y]`，对应网格上的一个点。

---

```python
rv = multivariate_normal(mean=mu_vec, cov=cov_mat)
pdf_2d = rv.pdf(pos)
```

* 创建一个二维正态分布对象 `rv`，使用之前定义的均值和协方差矩阵。
* 调用 `rv.pdf(pos)` 计算二维正态分布在每个网格点 `pos[i,j]` 上的概率密度值，结果 `pdf_2d` 是一个形状为 `(100, 100)` 的二维数组，存储对应的概率密度。

---

这段代码生成了一个二维平面上的点阵（网格），然后计算了二维高斯分布在这些点上的概率密度，用来绘制二维分布的等高线或热力图。


In [ ]:
# 创建图形和子图
fig = plt.figure(figsize=(12, 5))

# 左图：等高线图
ax1 = fig.add_subplot(1, 2, 1)
contour = ax1.contourf(X, Y, pdf_2d, cmap='viridis')
fig.colorbar(contour, ax=ax1)
ax1.set_title('2D Gaussian Contour')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')

# 右图：3D曲面图
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.plot_surface(X, Y, pdf_2d, cmap='viridis', edgecolor='none')
ax2.set_title('3D Gaussian Surface')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Density')

plt.tight_layout()
plt.show()


### 最大似然估计（MLE）和联合概率密度的关系

当我们做最大似然估计时，目标是**找到参数，使得在给定参数下，观测到的数据出现的概率最大**。换句话说，就是最大化数据的**似然函数**。

---

### 为什么用联合概率密度？

* 你有一组样本数据： $x = (x_1, x_2, \dots, x_n)$。
* 如果样本是独立同分布（i.i.d.），那么这组数据同时出现的概率（或概率密度）是各个样本点概率（密度）的乘积：

$$
p(x_1, x_2, \dots, x_n \mid \theta) = \prod_{i=1}^n p(x_i \mid \theta)
$$

这里 $\theta$ 是我们想估计的参数，比如正态分布的均值和方差。

* 这就是**联合概率密度**（联合分布），它表示在给定参数条件下，整个样本集合同时出现的概率（密度）。

---

### 最大化联合概率密度的理由

* 我们观察到整组数据 $x$，想找到让这组数据“最可能”出现的参数 $\theta$，自然要用**联合概率密度**作为衡量标准。
* 单个点的概率（密度）不够，必须考虑所有数据一起的可能性，这就是联合概率。

---

### 为什么通常用负对数似然？

* 乘积形式计算往往非常小且数值不稳定，
* 所以通常取对数（对数似然），把乘积变成求和，方便计算和优化：

$$
\log p(x_1, ..., x_n \mid \theta) = \sum_{i=1}^n \log p(x_i \mid \theta)
$$



# 四、梯度下降法求最大似然估计

## 1、负对数似然及梯度

In [ ]:
def nll_and_grads(x, mu, rho):
    """
    x: 样本
    mu: 均值参数
    rho: log(sigma)，保证sigma>0
    """
    n = x.size
    dif = x - mu
    S = np.sum(dif**2)
    exp_minus_2rho = np.exp(-2.0 * rho)
    L = n * rho + 0.5 * S * exp_minus_2rho  # 去掉常数项
    dL_dmu = (n * mu - np.sum(x)) * exp_minus_2rho
    dL_drho = n - S * exp_minus_2rho
    return L, dL_dmu, dL_drho

这段代码实现了\*\*一维正态分布负对数似然（Negative Log-Likelihood，NLL）\*\*及其对参数的梯度计算。参数是均值 `mu` 和一个经过变换的标准差参数 `rho`，其中 `rho = log(sigma)`，用来保证标准差 `sigma` 始终大于0。

---

### 输入参数

* `x`：样本数据，形状为一维数组，包含 `n` 个数据点。
* `mu`：正态分布的均值参数。
* `rho`：对数标准差参数，`rho = log(sigma)`，保证 `sigma = exp(rho)` 总是正数。

---

### 代码核心含义

```python
n = x.size
```

* 样本数量。

```python
dif = x - mu
S = np.sum(dif**2)
```

* 计算每个样本与均值的差值，然后计算平方差的和 $S = \sum_{i=1}^n (x_i - \mu)^2$。

```python
exp_minus_2rho = np.exp(-2.0 * rho)
```

* 计算 $e^{-2\rho} = \frac{1}{\sigma^2}$，因为 $\sigma = e^{\rho}$，所以 $\sigma^2 = e^{2\rho}$，逆即 $e^{-2\rho}$。

---

### 负对数似然函数 $L$

```python
L = n * rho + 0.5 * S * exp_minus_2rho  # 去掉常数项
```

* 标准正态分布的负对数似然（忽略常数项）为：

$$
L(\mu, \sigma) = n \log \sigma + \frac{1}{2 \sigma^2} \sum_{i=1}^n (x_i - \mu)^2
$$

* 这里用 `rho = log(sigma)`，所以

$$
L = n \cdot \rho + \frac{1}{2} S e^{-2\rho}
$$

---

### 梯度计算

```python
dL_dmu = (n * mu - np.sum(x)) * exp_minus_2rho
```

* 负对数似然对均值的梯度：

$$
\frac{\partial L}{\partial \mu} = \frac{1}{\sigma^2} \sum_{i=1}^n (\mu - x_i) = \frac{n\mu - \sum x_i}{\sigma^2}
$$

* 用 $e^{-2\rho} = \frac{1}{\sigma^2}$ 表示。

```python
dL_drho = n - S * exp_minus_2rho
```

* 负对数似然对 `rho` 的梯度：

$$
\frac{\partial L}{\partial \rho} = n - \frac{1}{\sigma^2} \sum_{i=1}^n (x_i - \mu)^2 = n - S e^{-2\rho}
$$

---

### 返回值

```python
return L, dL_dmu, dL_drho
```

* 返回负对数似然值 `L`，以及对均值 `mu` 和对数标准差参数 `rho` 的梯度。

---

### 总结

* 这个函数计算基于样本数据的一维正态分布的负对数似然（忽略常数项）及其对均值和对数标准差的梯度。
* 使用对数标准差 `rho` 保证标准差始终正值，有利于优化时参数的稳定性和数值安全。


## 2、梯度下降-基础版本

In [ ]:
def gradient_descent_mle_basic(x, mu0=None, rho0=None, lr_mu=0.01, lr_rho=0.01,
                               max_iters=5000, tol=1e-8):
    if mu0 is None:
        mu = np.mean(x) + 0.1 * np.std(x)
    else:
        mu = float(mu0)
    if rho0 is None:
        rho = np.log(np.std(x))
    else:
        rho = float(rho0)

    prev_L = None
    mu_hist, sigma_hist, loss_hist = [], [], []
    for it in range(1, max_iters+1):
        L, g_mu, g_rho = nll_and_grads(x, mu, rho)

        mu -= lr_mu * g_mu
        rho -= lr_rho * g_rho

        mu_hist.append(mu)
        sigma_hist.append(np.exp(rho))
        loss_hist.append(L)

        if prev_L is not None and abs(prev_L - L) < tol:
            break
        prev_L = L

    return mu, np.exp(rho), L, mu_hist, sigma_hist, loss_hist


### 函数作用

这个函数实现了基于**负对数似然（NLL）**的**梯度下降法**，用于估计一维正态分布的两个参数：均值 `mu` 和标准差（用对数参数 `rho = log(sigma)` 表示），使得模型拟合样本数据 `x`。

---

### 输入参数说明

* `x`: 样本数据（一维 numpy 数组）。
* `mu0`: 初始均值参数（默认用样本均值附近初始化）。
* `rho0`: 初始对数标准差参数（默认用样本标准差初始化）。
* `lr_mu`: 均值参数的学习率（更新步长）。
* `lr_rho`: 对数标准差参数的学习率。
* `max_iters`: 最大迭代次数。
* `tol`: 收敛阈值，当损失函数变化小于该值时停止迭代。

---

### 代码流程详解

```python
if mu0 is None:
    mu = np.mean(x) + 0.1 * np.std(x)
else:
    mu = float(mu0)
if rho0 is None:
    rho = np.log(np.std(x))
else:
    rho = float(rho0)
```

* 初始化参数。
* 如果用户没给初始值，默认：

  * `mu` 初始化为样本均值稍微偏移一点（避免刚开始梯度为0等情况）。
  * `rho` 初始化为样本标准差的对数（保证开始时的标准差合理）。

---

```python
prev_L = None
mu_hist, sigma_hist, loss_hist = [], [], []
```

* 记录之前的损失函数值（用于判断收敛）。
* 准备保存每次迭代的 `mu`、`sigma`（`exp(rho)`）和损失值，用于后续分析或画图。

---

```python
for it in range(1, max_iters+1):
    L, g_mu, g_rho = nll_and_grads(x, mu, rho)
```

* 进入迭代循环，最多执行 `max_iters` 次。
* 调用之前定义的 `nll_and_grads` 函数，计算当前参数下的负对数似然值 `L`，以及对 `mu` 和 `rho` 的梯度 `g_mu` 和 `g_rho`。

---

```python
mu -= lr_mu * g_mu
rho -= lr_rho * g_rho
```

* 根据梯度更新参数。
* 沿梯度负方向移动，使负对数似然（损失）减小。
* 学习率控制步长大小。

---

```python
mu_hist.append(mu)
sigma_hist.append(np.exp(rho))
loss_hist.append(L)
```

* 记录当前迭代的参数和损失值，方便查看收敛过程。

---

```python
if prev_L is not None and abs(prev_L - L) < tol:
    break
prev_L = L
```

* 判断是否收敛：

  * 如果当前损失与上一次变化小于阈值 `tol`，认为已经收敛，跳出循环。
* 更新 `prev_L` 以备下一轮比较。

---

```python
return mu, np.exp(rho), L, mu_hist, sigma_hist, loss_hist
```

* 返回最后得到的参数估计：

  * `mu`（均值）
  * `exp(rho)`（标准差）
  * `L`（最终负对数似然值）
  * 以及迭代过程中记录的参数和损失历史，方便分析和绘图。

---

该函数用梯度下降法，基于负对数似然函数，迭代更新参数 `mu` 和 `rho`，最终找到最优的正态分布参数拟合样本数据。简单、直观，适合理解和教学。



## 3、梯度下降-adam优化器版本

In [ ]:
def gradient_descent_mle_adam(x, mu0=None, rho0=None, lr_mu=0.01, lr_rho=0.01,
                              max_iters=5000, tol=1e-8):
    if mu0 is None:
        mu = np.mean(x) + 0.1 * np.std(x)
    else:
        mu = float(mu0)
    if rho0 is None:
        rho = np.log(np.std(x))
    else:
        rho = float(rho0)

    beta1, beta2 = 0.9, 0.999
    eps = 1e-8
    m_mu = 0.0; v_mu = 0.0
    m_rho = 0.0; v_rho = 0.0

    prev_L = None
    mu_hist, sigma_hist, loss_hist = [], [], []
    for it in range(1, max_iters+1):
        L, g_mu, g_rho = nll_and_grads(x, mu, rho)

        m_mu = beta1 * m_mu + (1 - beta1) * g_mu
        v_mu = beta2 * v_mu + (1 - beta2) * (g_mu ** 2)
        m_mu_hat = m_mu / (1 - beta1 ** it)
        v_mu_hat = v_mu / (1 - beta2 ** it)
        mu -= lr_mu * m_mu_hat / (np.sqrt(v_mu_hat) + eps)

        m_rho = beta1 * m_rho + (1 - beta1) * g_rho
        v_rho = beta2 * v_rho + (1 - beta2) * (g_rho ** 2)
        m_rho_hat = m_rho / (1 - beta1 ** it)
        v_rho_hat = v_rho / (1 - beta2 ** it)
        rho -= lr_rho * m_rho_hat / (np.sqrt(v_rho_hat) + eps)

        mu_hist.append(mu)
        sigma_hist.append(np.exp(rho))
        loss_hist.append(L)

        if prev_L is not None and abs(prev_L - L) < tol:
            break
        prev_L = L

    return mu, np.exp(rho), L, mu_hist, sigma_hist, loss_hist


## 4、训练并可视化

In [ ]:
x = data

# 解析解
mu_mle_closed = np.mean(x)
sigma2_mle_closed = np.mean((x - mu_mle_closed)**2)
sigma_mle_closed = np.sqrt(sigma2_mle_closed)

# 对比 Adam 和普通梯度下降的收敛曲线

# 普通梯度下降
mu_gd, sigma_gd, final_loss_gd, mu_hist_gd, sigma_hist_gd, loss_hist_gd = gradient_descent_mle_basic(
    x, lr_mu=0.0005, lr_rho=0.0005, max_iters=20000
)

# Adam
mu_adam, sigma_adam, final_loss_adam, mu_hist_adam, sigma_hist_adam, loss_hist_adam = gradient_descent_mle_adam(
    x, lr_mu=0.05, lr_rho=0.05, max_iters=20000
)

# 绘制对比
fig, axes = plt.subplots(3, 1, figsize=(8, 10))

# mu 收敛
axes[0].plot(mu_hist_gd, label="普通GD")
axes[0].plot(mu_hist_adam, label="Adam")
axes[0].axhline(mu_mle_closed, color='r', linestyle='--', label='解析解 μ')
axes[0].set_ylabel(r"$\mu$")
axes[0].set_title("均值参数收敛对比")
axes[0].legend()
axes[0].grid(alpha=0.3)

# sigma 收敛
axes[1].plot(sigma_hist_gd, label="普通GD")
axes[1].plot(sigma_hist_adam, label="Adam")
axes[1].axhline(sigma_mle_closed, color='r', linestyle='--', label='解析解 σ')
axes[1].set_ylabel(r"$\sigma$")
axes[1].set_title("标准差参数收敛对比")
axes[1].legend()
axes[1].grid(alpha=0.3)

# Loss 收敛
axes[2].plot(loss_hist_gd, label="普通GD")
axes[2].plot(loss_hist_adam, label="Adam")
axes[2].set_ylabel("Loss")
axes[2].set_xlabel("迭代次数")
axes[2].set_title("负对数似然收敛对比")
axes[2].grid(alpha=0.3)
axes[2].legend()

plt.tight_layout()
plt.show()

print((mu_gd, sigma_gd, final_loss_gd), (mu_adam, sigma_adam, final_loss_adam))

# 五、什么情况下不能用最大似然估计

## 1、数据分布形状未知怎么办？

### **如果分布未知，先要估计分布形状**

* 经典方法是用**非参数方法**估计密度，比如核密度估计（KDE），但这类方法一般不方便直接求参数梯度。
* 另一条路是**假设某个参数化的模型族**（例如高斯混合模型、神经网络生成模型等），用这些模型去拟合数据。
* 这时“参数”是模型内部参数，虽然不知道真实分布，但假设模型可拟合。

---

### **基于样本的优化（无模型参数梯度）**

* 如果没有明确的概率模型，梯度计算不能依赖概率密度函数的解析式。
* 可以用**基于样本的目标函数**，例如最小化某种损失函数，比如重构误差、对比损失等。
* 这样梯度是对损失函数的梯度，不是对概率密度函数的梯度。

---

### **使用概率无关的优化方法**

* 进化算法、强化学习、蒙特卡洛方法这类无梯度或近似梯度的优化方法可用。
* 例如使用**梯度估计方法**，比如强化学习中的策略梯度、REINFORCE算法，用采样和回报信号估计梯度。

---

### **现代机器学习中的方法**

* **生成模型（GAN、VAE）**：虽然分布未知，但模型学到一个参数化的生成分布，利用神经网络梯度求参数。
* **能量模型（Energy-Based Models）**：用能量函数替代概率密度，间接优化模型。
* **基于核方法或核嵌入的方法**，利用样本和核函数计算梯度。

---

### 5. **总结**

* **没有明确分布函数，不能直接写出负对数似然和梯度。**
* 需要假设或学习一个参数化的模型，或设计其他损失函数。
* 梯度来自模型参数对目标函数的导数，而不是直接对概率密度。
* 采样和近似梯度估计也是重要手段。


## 2、神经网络如何处理未知分布和参数？

**用神经网络参数化分布**

* 神经网络可以作为一个函数逼近器，学习一个从输入（例如随机噪声、条件变量）到数据空间的映射。
* 这种映射可以定义一个参数化的概率模型，比如生成对抗网络（GAN）、变分自编码器（VAE）等。
* 网络参数就是我们需要优化的“分布参数”，但它不是传统意义上的概率分布参数，而是网络权重和偏置。

**通过样本和目标函数学习参数**

* 你不需要事先知道数据的真实分布形式，只用样本数据训练神经网络。
* 通过定义合适的损失函数（例如GAN里的对抗损失，VAE里的变分下界，或者最大似然变分近似），网络通过**梯度下降**学习参数。
* 这里的梯度是通过**反向传播**自动计算的，无需显式写出概率密度梯度。

**损失函数起到“代理目标”的作用**

* 损失函数衡量神经网络输出与真实数据的“差异”或拟合程度。
* 通过不断最小化这个损失，神经网络参数迭代更新，间接学习数据的潜在分布。

**梯度计算和优化**

* 反向传播结合自动微分库（如PyTorch、TensorFlow）自动求梯度。
* 梯度用于梯度下降或其变体（Adam、RMSProp等）优化网络参数。
* 这里并不直接对概率密度求导，而是对损失函数求导。

---

### 举个简单例子

* 你有一组数据样本，但不知道分布。
* 建一个神经网络 $f_\theta(z)$，输入随机噪声 $z$，输出生成样本。
* 设计一个损失函数（比如GAN中的判别器判断生成样本和真实样本差异）。
* 用梯度下降优化网络参数 $\theta$，让生成样本越来越像真实样本。

---

### 总结

* **神经网络用参数化函数替代传统概率分布**，让“分布学习”成为参数学习问题。
* 通过定义可优化的损失函数和反向传播自动计算梯度。
* 不需要事先知道分布解析形式，只需样本数据即可进行训练。


# 六、正态分布概率密度函数推导

## 目标

推导一维正态分布概率密度函数的解析式：

$$
f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{(x - \mu)^2}{2\sigma^2}\right)
$$

---

## 1. 从高斯函数开始

假设概率密度函数形状满足：

$$
f(x) = A \exp\left(-B(x - \mu)^2\right)
$$

其中，$A > 0, B > 0$ 是常数，$\mu$ 是均值。

---

## 2. 要满足的条件

* **概率密度函数积分为1**：

$$
\int_{-\infty}^{+\infty} f(x) dx = 1
$$

* **均值是 $\mu$**，即

$$
E[X] = \int_{-\infty}^{+\infty} x f(x) dx = \mu
$$

* **方差是 $\sigma^2$**，即

$$
Var(X) = \int_{-\infty}^{+\infty} (x - \mu)^2 f(x) dx = \sigma^2
$$

---

## 3. 求积分确定 $A$

计算积分：

$$
\int_{-\infty}^{+\infty} A \exp(-B (x-\mu)^2) dx = 1
$$

变量替换：令 $y = x - \mu$，则

$$
\int_{-\infty}^{+\infty} A \exp(-B y^2) dy = 1
$$

利用高斯积分公式：

$$
\int_{-\infty}^{+\infty} e^{-a y^2} dy = \sqrt{\frac{\pi}{a}}, \quad a > 0
$$

得

$$
A \sqrt{\frac{\pi}{B}} = 1 \implies A = \sqrt{\frac{B}{\pi}}
$$

---

## 4. 确定 $B$ 使方差是 $\sigma^2$

计算方差：

$$
Var(X) = \int (x - \mu)^2 f(x) dx = \int y^2 A e^{-B y^2} dy
$$

利用积分结果（高斯二阶矩）：

$$
\int_{-\infty}^{+\infty} y^2 e^{-a y^2} dy = \frac{\sqrt{\pi}}{2 a^{3/2}}
$$

代入：

$$
Var(X) = A \frac{\sqrt{\pi}}{2 B^{3/2}} = \sqrt{\frac{B}{\pi}} \times \frac{\sqrt{\pi}}{2 B^{3/2}} = \frac{1}{2B}
$$

所以：

$$
\sigma^2 = \frac{1}{2B} \implies B = \frac{1}{2\sigma^2}
$$

---

## 5. 最终概率密度函数

带入 $A$ 和 $B$ 的表达式：

$$
A = \sqrt{\frac{1}{2\pi\sigma^2}} = \frac{1}{\sqrt{2\pi\sigma^2}}, \quad
B = \frac{1}{2\sigma^2}
$$

所以

$$
\boxed{
f(x) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)
}
$$



## 为什么用到高斯函数

为什么推导正态分布时，我们会先**假设概率密度函数是一个指数的平方型函数**，即

$$
f(x) = A \exp\left(-B(x - \mu)^2\right)
$$

这背后其实有几个深层的理由和历史背景：

---

### 1. 正态分布的定义和性质来源

* **正态分布最初是通过中心极限定理引入的**，它描述大量独立随机变量和的分布趋近于某种特定形式。
* 数学家发现，这种极限分布满足**对称、单峰、且在远离均值时快速衰减的性质**。

---

### 2. 高斯函数的自然形式

* 形如 $\exp(-B(x-\mu)^2)$ 的函数叫**高斯函数**，它的形状满足：

  * 对称于均值 $\mu$
  * 单峰，且远离 $\mu$ 时快速趋近于0
  * 二阶函数的指数保证了平滑且极易计算积分（解析性强）

---

### 3. 最大熵原理

* 如果已知一个连续随机变量的均值和方差，但对其它信息一无所知，**最大熵分布**（即在所有满足这两个约束的分布中熵最大的分布）就是正态分布。
* 这个最大熵分布自然对应于二次型指数函数。

---

### 4. 数学上的便利性

* 假设指数中是平方项，能方便求导和积分（高斯积分公式），方便验证概率密度函数的性质。
* 也是对称单峰分布的最简单解析表达。

---

### 5. 经验与实验

* 经验上许多自然现象（误差、噪声、测量误差）符合或近似符合这种平方指数形式。
* 所以假设先从这个形状入手，再确定系数 $A$ 和 $B$。

---

### 总结

* 这是一个经验、数学方便性和理论（最大熵、中心极限定理）共同作用的结果。
* 通过假设这种形式，我们能导出独一无二且满足概率密度性质的函数，即正态分布。

---



## 高斯函数的其它用途

高斯函数可不只是“拟合正态分布”的工具，它其实是数学、信号处理、物理、机器学习等领域的**超级多面手**。我帮你按领域整理一下：

---

## 1. 信号与图像处理

* **平滑滤波（Gaussian smoothing）**
  在图像处理中，用高斯卷积核去平滑图像，可以有效去除高频噪声，同时保留边缘的大致形状。
* **边缘检测（DoG / LoG）**
  先用高斯平滑，再求导（高斯差分、拉普拉斯-高斯），能提取出清晰的边缘。
* **抗混叠（Anti-aliasing）**
  在采样前用高斯低通滤波减少高频信号，避免混叠失真。

---

## 2. 概率与统计

* **核密度估计（Gaussian KDE）**
  用高斯核估计未知分布的概率密度，常用于数据分布可视化和非参数统计。
* **高斯混合模型（GMM）**
  把数据看成多个高斯分布的加权和，用于聚类、密度建模、异常检测。
* **贝叶斯推断**
  高斯分布是共轭分布，推导方便，很多后验分布推出来还是高斯。

---

## 3. 机器学习与模式识别

* **RBF核（径向基函数核）**
  支持向量机、核回归等方法中，高斯函数用作相似度度量，能把非线性问题映射到高维。
* **神经网络激活函数**
  RBF网络（Radial Basis Function Network）直接用高斯函数作为激活函数。
* **特征加权**
  在某些算法中，用高斯权重对距离较近的样本赋予更大影响。

---

## 4. 物理与工程

* **热传导方程的解**
  一维热传导的瞬时解就是高斯函数形状（热扩散曲线）。
* **波包分析**
  在量子力学中，高斯波包用于表示粒子位置和动量的概率分布。
* **光学衍射与成像**
  高斯光束模型描述激光的强度分布和传播特性。

---

## 5. 数学与其他

* **卷积的好性质**
  两个高斯函数卷积还是高斯，方差相加——这在信号叠加、概率建模里非常方便。
* **傅里叶变换**
  高斯函数的傅里叶变换还是高斯（自相似性），在频域分析中很重要。
* **权重函数**
  在数值积分、加权最小二乘等地方，高斯函数常用来给“中间值”更多权重，远处值权重逐渐衰减。

---

✅ 小结：
高斯函数之所以这么万能，核心原因是：

1. 数学形式简洁，推导方便
2. 平滑且无限可微，适合近似和优化
3. 在卷积和变换下稳定（自相似）
